In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import importlib

In [3]:
# helper function ugh
def load_dab(alias: str = None):
    """Handle importing and reloading modules when in colab"""
    import requests, importlib, sys
    if not filename.endswith(".py"): filename += ".py"
    
    if 'google.colab' in sys.modules:
        if '/content/' not in sys.path:
            sys.path.insert(0, '/content/')

        url = f"https://raw.githubusercontent.com/BenAF002/dabble_bot_v2_dev/refs/heads/main/dabble_bot"
        code = requests.get(url).text

        # write to /content for colab
        destination = f"/content/dabble_bot"
        with open(destination, "w") as f:
            f.write(code)

    # handle module reloading
    module_name = filename[:-3] # Strip the .py extension
    if alias in globals():
        importlib.reload(globals()[alias])
    elif module_name in globals():
        importlib.reload(globals()[module_name])
    else:
        if alias is not None:
            globals()[alias] = importlib.import_module(module_name)
        else:
            globals()[module_name] = importlib.import_module(module_name)

In [5]:
load_py("phonemes_bert.py", alias="pb")
load_py("phonemes_dataset.py", alias="pds")

NameError: name 'load_py' is not defined

In [3]:
wd = phon.dataset.Words()
train, test = wd.train_test_split(0.8)

train.__len__(), test.__len__()

(93131, 23283)

## BERT Encoder

In [16]:
importlib.reload(dab)
bert = dab.phonemes_bert.PhonemeBERT(n_emb=64, exp=4, n_heads=2, n_blocks=2, dropout=0.0, pool='cls', mask_attn=True)

print(f"n params: {sum(p.numel() for p in bert.parameters()):,.0f}")
if bert.device == 'cuda': 
    bert.to('cuda')
    bert = torch.compile(bert)


n params: 114,610


In [17]:
# try training
optimizer = torch.optim.AdamW(bert.parameters(), lr=6e-4)

dab.train.training_loop(
    model=bert, 
    train_data=train, 
    test_data=test,
    optimizer=optimizer,
    batch_size=256,
    n_epochs=20, 
)

Epoch 1/20 - Train Loss: 0.3727, Test Loss: 0.2297 | ETA: 1 min, 28 sec
Epoch 2/20 - Train Loss: 0.2111, Test Loss: 0.1975 | ETA: 1 min, 27 sec
Epoch 3/20 - Train Loss: 0.1884, Test Loss: 0.1812 | ETA: 1 min, 24 sec
Epoch 4/20 - Train Loss: 0.1732, Test Loss: 0.1683 | ETA: 1 min, 18 sec
Epoch 5/20 - Train Loss: 0.1616, Test Loss: 0.1577 | ETA: 1 min, 12 sec
Epoch 6/20 - Train Loss: 0.1533, Test Loss: 0.1508 | ETA: 1 min, 8 sec
Epoch 7/20 - Train Loss: 0.1465, Test Loss: 0.1448 | ETA: 1 min, 2 sec
Epoch 8/20 - Train Loss: 0.1401, Test Loss: 0.1390 | ETA: 0 min, 57 sec
Epoch 9/20 - Train Loss: 0.1345, Test Loss: 0.1326 | ETA: 0 min, 52 sec
Epoch 10/20 - Train Loss: 0.1289, Test Loss: 0.1277 | ETA: 0 min, 48 sec
Epoch 11/20 - Train Loss: 0.1243, Test Loss: 0.1232 | ETA: 0 min, 43 sec
Epoch 12/20 - Train Loss: 0.1202, Test Loss: 0.1206 | ETA: 0 min, 38 sec
Epoch 13/20 - Train Loss: 0.1169, Test Loss: 0.1169 | ETA: 0 min, 33 sec
Epoch 14/20 - Train Loss: 0.1138, Test Loss: 0.1141 | ETA: 0 m

In [ ]:
def cosine_sim(a, b):
    if isinstance(a, torch.Tensor):
        a = a.cpu().detach().numpy()
    if isinstance(b, torch.Tensor):
        b = b.cpu().detach().numpy()
    return np.dot(a, b.T) / (np.linalg.norm(a) * np.linalg.norm(b))

def mse(a, b):
    return np.mean((a - b)**2)

In [ ]:
def wenc(word):
    res = [wd.ctoi[c] for c in word] + [0] * (15 - len(word))  # right pad with zeros
    res = torch.tensor(res, dtype=torch.int32).view(1, -1)
    res = res.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
    return res

def wemb(word):
    return wd.mappings[word]

In [ ]:
cosine_sim(bert(wenc('tough')), wemb('though')), cosine_sim(bert(wenc('though')), wemb('tough'))

(array([0.21800941], dtype=float32), array([0.23082455], dtype=float32))

## GRU Decoder

In [4]:
importlib.reload(phon.decoder)

gru = phon.decoder.DecoderGRU(input_size=(64+50), hidden_size=256)
gru.to(torch.device('cuda'))
gru = torch.compile(gru)
params_devices = {p.device for p in gru.parameters()}
print("model param devices:", params_devices)
print(f"n params: {sum(p.numel() for p in gru.parameters()):,.0f}")

model param devices: {device(type='cuda', index=0)}
n params: 295,968


In [5]:
optimizer = torch.optim.AdamW(gru.parameters(), lr=6e-4)

importlib.reload(phon.train)
phon.train.training_loop(
    model=gru, 
    train_data=train, 
    test_data=test, 
    optimizer=optimizer,
    batch_size=256,
    n_epochs=1, 
)

torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])


W1216 19:30:05.520000 22364 site-packages\torch\_inductor\utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.int32
torch.Size([256, 50]) torch.Size([256, 18])
torch.float32 torch.

In [7]:
dl = DataLoader(test, batch_size=32, shuffle=True)
yb, xb = next(iter(dl))
yb, xb = yb.to(gru.device), xb.to(gru.device)

for i in range(10):
    print(
        wd.dec(yb[i]), ' : ',
        wd.dec(gru.predict(xb[i].unsqueeze(1).T)[0])
    )

guzek  :  gueg
detach  :  datidt
cavagnaro  :  arkowango
creech  :  kurie
foose  :  faues
boasting  :  boottyming
discussed  :  decesfud
fredericksen  :  vertircon
knickknacks  :  cnouricac
washington  :  awithon


## Transformer Decoder